In [1]:
import pyodbc
import os
import logging 
import random
from datetime import datetime, timedelta

logging.basicConfig(format='%(levelname)s: %(asctime)s => %(message)s', level=logging.INFO)

connstr = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=nas.home.michaelfudge.com,11433;DATABASE=master;UID=sa;PWD=wXp55!929ds'

def runSql(sql):
    with pyodbc.connect(connstr, autocommit=True) as conn:
        commands = sql.split("GO\n")
        for command in commands:
            conn.execute(command)

def getCursor():
    with pyodbc.connect(connstr, autocommit=False) as conn:
        return conn.cursor()

        
def readFileContents(filespec):
    with open (filespec,'r') as f:
        return f.read()
    
def initializeDb(dbname, drop=True):
    script_path = os.path.join('dbname','sql')
    files = os.listdir(script_path)
    downscript = [file for file in files if file.startswith("dn-0")][0]
    if drop:
        print(f"FILE {downscript}")
        sql = readFileContents(os.path.join(script_path, downscript))
        print(sql)
        runSql(sql)
    upscripts = [file for file in files if file.startswith("up-")]
    upscripts.sort()
    for script in upscripts:
        sql = readFileContents(os.path.join(script_path, script))
        print(f"FILE {script}")
        print(sql)
        runSql(sql)
        